In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

BASE_URL = 'https://www.ambitionbox.com/jobs/search?designation=data-scientist'
HEADERS = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

job_titles = []
job_companies = []
job_locations =[]
job_experience = []
job_salary = []
job_skills = []

def get_soup(url):
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        else:
            print(f"Error: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")   

def get_job_details(soup):
    job_info_cards = soup.find_all('div', class_='jobsInfoCardCont')
    for job_info in job_info_cards:
        job_title = job_info.find('h2').text.strip()
        job_company = job_info.find('p', class_='company').text.strip()
        
        job_basic_info = job_info.find('div', class_='job-basic-info')
        job_details = job_basic_info.find_all('div', class_='entity')
        
        if len(job_details) == 4:
            loc_list = job_details[2]['title'].split(',')
            for loc in loc_list:
                add_job_details(job_title, job_company, job_details[0]['title'], job_details[1]['title'], loc.strip(), job_details[3]['title'])            
        elif len(job_details) == 3:
            loc_list = job_details[1]['title'].split(',')
            for loc in loc_list:
                add_job_details(job_title, job_company, job_details[0]['title'], 'Not Available', loc.strip(), job_details[2]['title'])
        elif len(job_details) == 2:
            loc_list = job_details[0]['title'].split(',')
            for loc in loc_list:
                add_job_details(job_title, job_company, 'Not Available', 'Not Available', loc.strip(), job_details[1]['title'])


def add_job_details(title, company, exp, sal, loc, skl):
    job_titles.append(title)
    job_companies.append(company)
    job_experience.append(exp)
    job_salary.append(sal)
    job_locations.append(loc)
    job_skills.append(skl)

for page in range(1, 11):
    req_url = f'{BASE_URL}&page={page}'
    soup = get_soup(req_url)
    get_job_details(soup)
    
df = pd.DataFrame({'Title': job_titles, 'Company': job_companies, 'Experience': job_experience, 'Salary': job_salary, 'Location': job_locations, 'Skills': job_skills})
df.to_csv('jobs.csv', index=False)
